In [ ]:
import pandas as pd
import numpy as np
import string
import json
import random
import datetime

In [ ]:
## Static Config
locations = ["woreda", "kebele"]
latlong = ["latitude", "longitude"]
charts = [
    {
        "name": "Communitites ODF",
        "column" : "ODF Status",
        "action": "percentage",
        "type": "bar",
        "value": "ODF",
        "and": None,
        "and_value": None,
    },
    {
        "name":  "Communities Triggered but Not Yet ODF",
        "column" : "ODF Status",
        "action": "percentage",
        "type": "bar",
        "value": "OD",
        "and": "Date Triggered",
        "and_value": "not null" 
    }
]

## Table config
table = [
    {
        "name": "Summary Data",
        "type": "summary",
        "indicators": [
            {
                "name": "Total Number of Communities",
                "column": "No. of HHs",
                "action": "sum",
                "type": "number",
                "value": None,
                "and": None,
                "and_value": None,
            },
            {
                "name": "% of Communities Triggered but not yet ODF",
                "column": "ODF Status",
                "action": "percentage",
                "type": "number",
                "value": "OD",
                "and": "Date Triggered",
                "and_value": "not null"
            },
            {
                "name": "% of Communities that are ODF",
                "column": "ODF Status",
                "action": "percentage",
                "type": "number",
                "value": "ODF",
                "and": None,
                "and_value": None,
            },
        ],
    },
    {
        "name": "Detailed Data for Community",
        "type": "detail",
        "indicators": [
            {
                "name": "Number of Households",
                "column": "No. of HHs",
                "action": "select",
                "type": "number",
                "value": None,
                "and": None,
                "and_value": None,
            },
            {
                "name": "Number of latrines",
                "column": "Number of Latrines",
                "action": "select",
                "type": "number",
                "value": None,
                "and": None,
                "and_value": None,
            },
            {
                "name": "Date of Triggering",
                "column": "Date Triggered",
                "action": "select",
                "type": "date",
                "value": None,
                "and": None,
                "and_value": None,
            }
        ],
    },
]

## Marker Detail
marker = {
    "name" : "Village",
}

In [ ]:
df = pd.read_csv('eth_clts_data.csv')

In [ ]:
## Delete Column with no name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
## Fill Column
# default_column = ["Woreda", "Kebele", "Village", "Latitude", "Longitude"]
df["No. of HHs"] = df["No. of HHs"].apply(lambda x: random.randint(20, 200))

In [ ]:
## Fill Column
def fillLatrines(x):
    maxVal = x["No. of HHs"]
    return random.randint(random.choice([20, maxVal]), maxVal)

df["Number of Latrines"] = df.apply(fillLatrines, axis=1)

In [ ]:
## Fill Column
def fillTrigger(x):
    startdate=datetime.date(2018,1,1)
    date=startdate+datetime.timedelta(random.randint(300,730))
    if (x["No. of HHs"] == x["Number of Latrines"]):
        return date
    if (x["Number of Latrines"] <= random.randint((x["No. of HHs"] - 20), x["No. of HHs"])):
        return date
    return None

df["Date Triggered"] = df.apply(fillTrigger, axis=1)

In [ ]:
## Fill Column
def fillStatus(x):
    if (x["No. of HHs"] == x["Number of Latrines"] and x["Date Triggered"] != None):
        return "ODF"
    return "OD"

df["ODF Status"] = df.apply(fillStatus, axis=1)

In [ ]:
## Drop Columns
df = df.drop(columns=['Open Defecation Free'])

In [ ]:
### Fill Empty Integer Values
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num = df.select_dtypes(include=numerics).drop(columns=['Latitude', 'Longitude'])
df_num = df_num.fillna(0.0).astype(np.int32)
df[list(df_num)] = df_num

In [ ]:
### Replace Empty String Values
df_str = df.select_dtypes(include=['object']).fillna("")
df[list(df_str)] = df_str

In [ ]:
## Generate Settings 
chars =list(string.ascii_uppercase)
chars_col = chars + [x+y for x in chars for y in chars]
keyname = lambda x,y: {a:y[b] if (a != 'data_point_id') else 'data_point_id' for b, a in enumerate(x)}

columns_length = len(list(df))
column_names = list(df)

In [ ]:
### Generate configs
index = chars_col[:columns_length]
configs = keyname(index, column_names)

marker_config = {}
loc_config = {}
latlong_config = {}
for key in configs:
    if(configs[key].lower() in locations):
        loc_config.update({configs[key].lower(): key})
    if(configs[key].lower() in latlong):
        latlong_config.update({configs[key].lower(): key})
    if(configs[key].lower() == marker['name'].lower()):
        marker_config = { "name" : key}
        
        
chart_config = []
for ch in charts:
    column = None
    andColumn = None
    for key in configs:
        if configs[key].lower() == ch['column'].lower():
            column = key
        if ch['and'] != None and configs[key].lower() == ch['and'].lower():
            andColumn = key
    chart_config.append({
        "name": ch['name'],
        "column": column,
        "action": ch['action'],
        "type": ch['type'],
        "value": ch['value'],
        "and": andColumn,
        "and_value": ch['and_value'],
    })





#### Table
table_config = []
for tb in table:
    tb_indicators = []
    for ind in tb['indicators']:
        column = None
        andColumn = None;
        for key in configs:
            if configs[key].lower() == ind["column"].lower():
                column = key
            if ind["and"] != None and configs[key].lower() == ind['and'].lower():
                andColumn = key
        tb_indicators.append({ 
            "column": column, 
            "name": ind['name'], 
            "action": ind['action'], 
            "type": ind['type'],
            "value": ind['value'],
            "and": andColumn,
            "and_value": ind['and_value'],
        })
    table_config.append({
        "name": tb['name'],
        "type": tb['type'],
        "indicators": tb_indicators
    })


configs['marker'] = marker_config 
configs['charts'] = chart_config
configs['locations'] = loc_config
configs['latlong'] = latlong_config
configs['table'] = table_config

In [ ]:
configs

In [ ]:
### Replace Dataset Columns
df.columns = index

In [ ]:
configs

In [ ]:
data = list(df.T.to_dict().values())
data[:2]

In [ ]:
### to json
df.to_json('./eth_clts_data.json', orient="records", indent=2)

with open('./eth_clts_config.json', 'w') as outfile:
    json.dump(configs, outfile)